# This notebook will be used for IBM Data Science Capstone Project

In [553]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [554]:
import requests
from bs4 import BeautifulSoup
! pip install html5lib
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [555]:
#Accessing the Data
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 


# This step parses through the data saving the values of postalcode, borough and neighboorhod

In [556]:
#Collecting the Data
PostalCode=[]
Borough=[]
Neighborhood=[]
rows=[]
cnum=1 # parsing by column number since its only 3 columns 
for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            if cnum==1:
                PostalCode.append(cell.string)
                cnum=2
            elif cnum==2 :
                if cell.string=='Not assigned\n':
                    Borough.append(None)
                    cnum=3
                else:
                    Borough.append(cell.string)
                    cnum=3
            else:
                Neighborhood.append(cell.string)
                cnum=1

# Now I merge the data collected from the previous step and produce a Dataframe 

In [557]:
#Merging the Data
code=[PostalCode,Borough,Neighborhood]
codes=pd.DataFrame(code)
codes=codes.transpose()

In [558]:
# Formating data into dataframes with a coloumn 
#tor refers to toronto
tor=pd.DataFrame(data=codes)
tor.columns=['PostalCode','Borough','Neighborhood']
tor.head()

PostalCode            Borough                Neighborhood
0       M1A
               None               Not assigned

1       M2A
               None               Not assigned

2       M3A
        North York
                  Parkwoods

3       M4A
        North York
           Victoria Village

4       M5A
  Downtown Toronto
  Regent Park, Harbourfront

# Cleaning the data from any unnecessary information

In [559]:
#removing any none values
tor.dropna(subset=['Borough'],axis=0,inplace=True)
tor=tor.reset_index(drop=True)
tor.head()

PostalCode            Borough                                  Neighborhood
0       M3A
        North York
                                    Parkwoods

1       M4A
        North York
                             Victoria Village

2       M5A
  Downtown Toronto
                    Regent Park, Harbourfront

3       M6A
        North York
             Lawrence Manor, Lawrence Heights

4       M7A
  Downtown Toronto
  Queen's Park, Ontario Provincial Government

In [560]:
tor.shape

(103, 3)

## loading Latitude and Longitude coordinates

In [561]:
latlong=pd.read_csv('http://cocl.us/Geospatial_data')

In [562]:
latlong.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [563]:
tor=tor.sort_values('PostalCode')
latlong.columns=['PostalCode','Latitude','Longitude']

latlong.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [564]:
#removing unnecessary formatting
tor=tor.replace('\n','', regex=True)
tor.head()


PostalCode      Borough                            Neighborhood
6         M1B  Scarborough                          Malvern, Rouge
12        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
18        M1E  Scarborough       Guildwood, Morningside, West Hill
22        M1G  Scarborough                                  Woburn
26        M1H  Scarborough                               Cedarbrae

In [565]:
#casting both dataframes to type string
tor['PostalCode'] = tor['PostalCode'].astype(str)
latlong['PostalCode'] = latlong['PostalCode'].astype(str)
latlong.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [566]:
#merging both dataframes
tor = pd.merge(tor,latlong)
tor.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

# Map of Toronto with Neighborhoods marked

In [625]:
address = 'Toronto, CA'
# using geopy to find latitude and longitude of Toronto
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
#get the Latitude and longitude of toronto
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor['Latitude'], tor['Longitude'], tor['Borough'], tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [570]:
tor['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

# Lets find the Boroughs with Toronto in their name 

In [627]:
Toronto_data = tor[tor['Borough'].str.contains('Toronto',regex=False)].reset_index(drop=True)
Toronto_data['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [572]:
#lets map these Boroughs
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

# create map of toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers of the toronto named boroughs to map 
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

The geograpical coordinate of North York are 43.6534817, -79.3839347.


# Foursquare 

In [ ]:
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [629]:
#first neighborhood of Toronto
Toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [575]:
# coordinates of The Beaches
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


# top 100 venues

In [576]:
LIMIT=100
radius=500

##Define URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [577]:
#get the venues names
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eeab89747b43d00238e71bb'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [578]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [579]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name         categories        lat        lng
0                   Glen Manor Ravine              Trail  43.676821 -79.293942
1  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
2                 Grover Pub and Grub                Pub  43.679181 -79.297215
3                       Upper Beaches       Neighborhood  43.680563 -79.292869

# Explore Neighborhoods

In [580]:
#combine previous steps into a function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [581]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [582]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1627, 7)


Neighborhood  Neighborhood Latitude  \
0                   The Beaches              43.676357   
1                   The Beaches              43.676357   
2                   The Beaches              43.676357   
3                   The Beaches              43.676357   
4  The Danforth West, Riverdale              43.679557   

   Neighborhood Longitude                               Venue  Venue Latitude  \
0              -79.293031                   Glen Manor Ravine       43.676821   
1              -79.293031  The Big Carrot Natural Food Market       43.678879   
2              -79.293031                 Grover Pub and Grub       43.679181   
3              -79.293031                       Upper Beaches       43.680563   
4              -79.352188                       MenEssentials       43.677820   

   Venue Longitude     Venue Category  
0       -79.293942              Trail  
1       -79.297734  Health Food Store  
2       -79.297215                Pub  
3       -79.292869       Neighborhood  
4       -79.351265     Cosmetics Shop

In [583]:
Toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            58   
Brockton, Parkdale Village, Exhibition Place                           24   
Business reply mail Processing Centre, South Ce...                     19   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     67   
Christie                                                               17   
Church and Wellesley                                                   80   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           16   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          23   
India Bazaar, The Beaches West                                         23   
Kensington Market, Chinatown, Grange Park                              61   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               44   
Moore Park, Summerhill East                                             1   
North Toronto West,  Lawrence Park                                     19   
Parkdale, Roncesvalles                                                 15   
Queen's Park, Ontario Provincial Government                            32   
Regent Park, Harbourfront                                              44   
Richmond, Adelaide, King                                               93   
Rosedale                                                                4   
Roselawn                                                                4   
Runnymede, Swansea                                                     38   
St. James Town                                                         78   
St. James Town, Cabbagetown                                            45   
Stn A PO Boxes                                                         97   
Studio District                                                        40   
Summerhill West, Rathnelly, South Hill, Forest ...                     17   
The Annex, North Midtown, Yorkville                                    20   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         35   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             58   
Brockton, Parkdale Village, Exhibition Place                            24   
Business reply mail Processing Centre, South Ce...                      19   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      67   
Christie                                                                17   
Church and Wellesley                                                    80   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              35   
Davisville North                    

# Analyze each neighborhood

In [584]:
# one hot encoding to change it to the following table and take percentage
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Theme Restaurant  Toy / Game Store  Trail  \
0             0  ...                 0                 0      1   
1             0  ...                 0                 0      0   
2             0  ...                 0                 0      0   
3             0  ...                 0                 0      0   
4             0  ...                 0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  Women's Store  
0                      0         0          0              0  
1                      0         0          0              0  
2                      0         0          0              0  
3                      0         0          0              0  
4                      0         0          0              0  

[5 rows x 240 columns]

In [585]:
Toronto_onehot.shape

(1627, 240)

In [586]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1        Brockton, Parkdale Village, Exhibition Place     0.000000   
2   Business reply mail Processing Centre, South C...     0.052632   
3   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                  Central Bay Street     0.014925   
5                                            Christie     0.000000   
6                                Church and Wellesley     0.025000   
7                      Commerce Court, Victoria Hotel     0.000000   
8                                          Davisville     0.000000   
9                                    Davisville North     0.000000   
10                       Dufferin, Dovercourt Village     0.000000   
11             First Canadian Place, Underground city     0.000000   
12    Forest Hill North & West, Forest Hill Road Park     0.000000   
13                           Garden District, Ryerson     0.000000   
14  Harbourfront East, Union Station, Toronto Islands     0.000000   
15                      High Park, The Junction South     0.000000   
16                     India Bazaar, The Beaches West     0.000000   
17          Kensington Market, Chinatown, Grange Park     0.000000   
18                                      Lawrence Park     0.000000   
19                           Little Portugal, Trinity     0.022727   
20                        Moore Park, Summerhill East     0.000000   
21                 North Toronto West,  Lawrence Park     0.052632   
22                             Parkdale, Roncesvalles     0.000000   
23        Queen's Park, Ontario Provincial Government     0.031250   
24                          Regent Park, Harbourfront     0.022727   
25                           Richmond, Adelaide, King     0.000000   
26                                           Rosedale     0.000000   
27                                           Roselawn     0.000000   
28                                 Runnymede, Swansea     0.026316   
29                                     St. James Town     0.000000   
30                        St. James Town, Cabbagetown     0.000000   
31                                     Stn A PO Boxes     0.010309   
32                                    Studio District     0.025000   
33  Summerhill West, Rathnelly, South Hill, Forest...     0.000000   
34                The Annex, North Midtown, Yorkville     0.000000   
35                                        The Beaches     0.000000   
36                       The Danforth West, Riverdale     0.023256   
37           Toronto Dominion Centre, Design Exchange     0.000000   
38                     University of Toronto, Harbord     0.028571   

    Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0              0.0000   0.0000              0.0000        0.0000   
1              0.0000   0.0000              0.0000        0.0000   
2              0.0000   0.0000              0.0000        0.0000   
3              0.0000   0.0625              0.0625        0.0625   
4              0.0000   0.0000              0.0000        0.0000   
5              0.0000   0.0000              0.0000        0.0000   
6              0.0125   0.0000              0.0000        0.0000   
7              0.0000   0.0000              0.0000        0.0000   
8              0.0000   0.0000              0.0000        0.0000   
9              0.0000   0.0000              0.0000        0.0000   
10             0.0000   0.0000              0.0000        0.0000   
11             0.0000   0.0000              0.0000        0.0000   
12             0.0000   0.0000              0.0000        0.0000   
13             0.0000   0.0000              0.0000        0.0000   
14             0.0000   0.0000              0.0000        0.0000   
15             0.0000   0.0000              0.0000        0.0000   
16             0.0000   0.0000              0.0000        0.0000   
17             0

In [587]:
#top 5 venues printed
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[NorthYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2   Cheese Shop  0.03
3        Bakery  0.03
4    Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1  Performing Arts Venue  0.08
2            Coffee Shop  0.08
3         Breakfast Spot  0.08
4                    Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.05
2                   Spa  0.05
3  Gym / Fitness Center  0.05
4         Garden Center  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Airport Service  0.19
1   Airport Lounge  0.12
2            Plane  0.06
3  Harbor / Marina  0.06
4         Boutique  0.06


----Central Bay Street----
                 venue  freq
0      

In [588]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [615]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue  3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar             Restaurant    Seafood Restaurant   
1        Breakfast Spot  Performing Arts Venue           Coffee Shop   
2           Yoga Studio            Pizza Place            Smoke Shop   
3        Airport Lounge            Coffee Shop       Harbor / Marina   
4    Italian Restaurant         Sandwich Place   Japanese Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0              Beer Bar                Bakery           Cheese Shop   
1          Climbing Gym         Burrito Place            Restaurant   
2            Skate Park               Brewery         Burrito Place   
3              Boutique         Boat or Ferry   Rental Car Location   
4                  Café           Salad Place      Department Store   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
0                  Café                 Diner              Shopping Mall  
1    Italian Restaurant          Intersection                    Stadium  
2            Restaurant      Recording Studio                 Comic Shop  
3                 Plane      Sculpture Garden                    Airport  
4          Dessert Shop       Thai Restaurant  Middle Eastern Restaurant

In [616]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [617]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

PostalCode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0                   Pub     Health Food Store   
1 -79.352188               0      Greek Restaurant           Coffee Shop   
2 -79.315572               0                  Park  Fast Food Restaurant   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               3                  Park           Swim School   

  3rd Most Common Venue   4th Most Common Venue 5th Most Common Venue  \
0                 Trail           Women's Store   Distribution Center   
1    Italian Restaurant  Furniture / Home Store        Ice Cream Shop   
2                   Gym                     Pub               Brewery   
3                Bakery               Gastropub               Brewery   
4              Bus Line        Department Store     Electronics Store   

         6th Most Common Venue 7th Most Common Venue    8th Most Common Venue  \
0                 Dessert Shop    Dim Sum Restaurant                    Diner   
1                    Bookstore    Frozen Yogurt Shop              Yoga Studio   
2                 Liquor Store        Sandwich Place            Burrito Place   
3          American Restaurant           Yoga Studio  Comfort Food Restaurant   
4  Eastern European Restaurant   Dumpling Restaurant               Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
0        Discount Store       Doner Restaurant  
1     Indian Restaurant                    Pub  
2    Italian Restaurant             Restaurant  
3    Seafood Restaurant         Sandwich Place  
4      Doner Restaurant                Dog Run

In [618]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters


In [619]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0       East Toronto               0                   Pub   
1       East Toronto               0      Greek Restaurant   
2       East Toronto               0                  Park   
3       East Toronto               0                  Café   
5    Central Toronto               0                 Hotel   
6    Central Toronto               0           Coffee Shop   
7    Central Toronto               0           Pizza Place   
9    Central Toronto               0                   Pub   
11  Downtown Toronto               0           Coffee Shop   
12  Downtown Toronto               0           Coffee Shop   
13  Downtown Toronto               0           Coffee Shop   
14  Downtown Toronto               0        Clothing Store   
15  Downtown Toronto               0           Coffee Shop   
16  Downtown Toronto               0           Coffee Shop   
17  Downtown Toronto               0           Coffee Shop   
18  Downtown Toronto               0           Coffee Shop   
19  Downtown Toronto               0           Coffee Shop   
20  Downtown Toronto               0           Coffee Shop   
21  Downtown Toronto               0           Coffee Shop   
24   Central Toronto               0        Sandwich Place   
25  Downtown Toronto               0                  Café   
26  Downtown Toronto               0                  Café   
27  Downtown Toronto               0       Airport Service   
28  Downtown Toronto               0           Coffee Shop   
29  Downtown Toronto               0           Coffee Shop   
30  Downtown Toronto               0         Grocery Store   
31      West Toronto               0              Pharmacy   
32      West Toronto               0                   Bar   
33      West Toronto               0                  Café   
34      West Toronto               0    Mexican Restaurant   
35      West Toronto               0             Gift Shop   
36      West Toronto               0           Pizza Place   
37  Downtown Toronto               0           Coffee Shop   
38      East Toronto               0    Light Rail Station   

   2nd Most Common Venue  3rd Most Common Venue   4th Most Common Venue  \
0      Health Food Store                  Trail           Women's Store   
1            Coffee Shop     Italian Restaurant  Furniture / Home Store   
2   Fast Food Restaurant                    Gym                     Pub   
3            Coffee Shop                 Bakery               Gastropub   
5                    Gym                   Park          Breakfast Spot   
6         Clothing Store            Yoga Studio     Sporting Goods Shop   
7           Dessert Shop         Sandwich Place                     Gym   
9            Coffee Shop    American Restaurant             Supermarket   
11           Pizza Place                 Bakery                  Market   
12   Japanese Restaurant       Sushi Restaurant              Restaurant   
13                  Park                 Bakery                     Pub   
14           Coffee Shop         Cosmetics Shop     Japanese Restaurant   
15                  Café           Cocktail Bar               Gastropub   
16          Cocktail Bar             Restaurant      Seafood Restaurant   
17    Italian Restaurant         Sandwich Place     Japanese Restaurant   
18                  Café             Restaurant                   Hotel   
19              Aquarium                  Hotel                    Café   
20                 Hotel                   Café              Restaurant   
21                  Café             Restaurant                   Hotel   
24                  Café            Coffee Shop                    Park   
25                Bakery              Bookstore                     Bar   
26           Coffee Shop                 Bakery   Vietnamese Restaurant   
27        Airport Lounge            Coffee Shop         Harbor / Marina   
28                  Café                  

In [620]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
8  Central Toronto               1          Tennis Court   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
8         Women's Store      Department Store  Ethiopian Restaurant   

  5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
8     Electronics Store  Eastern European Restaurant   Dumpling Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
8            Donut Shop      Doner Restaurant                Dog Run

In [621]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
10  Downtown Toronto               2                  Park   
23   Central Toronto               2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10            Playground                 Trail         Deli / Bodega   
23         Jewelry Store                 Trail      Sushi Restaurant   

   5th Most Common Venue        6th Most Common Venue  \
10     Electronics Store  Eastern European Restaurant   
23         Women's Store             Department Store   

          7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
10          Dumpling Restaurant            Donut Shop      Doner Restaurant   
23  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   10th Most Common Venue  
10                Dog Run  
23       Doner Restaurant

In [622]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
4  Central Toronto               3                  Park   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
4           Swim School              Bus Line      Department Store   

  5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
4     Electronics Store  Eastern European Restaurant   Dumpling Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
4            Donut Shop      Doner Restaurant                Dog Run

In [623]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
22  Central Toronto               4          Home Service   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
22           Music Venue                Garden         Women's Store   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
22          Dessert Shop    Dim Sum Restaurant                 Diner   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
22        Discount Store   Distribution Center          Deli / Bodega